<a href="https://colab.research.google.com/github/avanishbatkulia/Project-mebourne-automated/blob/main/project_Melbourne(31_01).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feedparser pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=b2cfd73ec8dcf23b23e2963aeb9090e4d8d1738b09affdf34b3e9d26684d5a61
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [2]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.9
    Uninstalling openai-1.59.9:
      Successfully uninstalled openai-1.59.9


In [3]:
!pip install transformers

In [4]:
import feedparser
import pandas as pd
from datetime import datetime


In [ ]:
rss_feeds = [
    'https://www.theage.com.au/rss/national/victoria.xml',
    'https://www.9news.com.au/rss',
    'https://www.theguardian.com/rss',
    'https://7news.com.au/feed'
]


Parse the RSS Fees

In [ ]:
#Define a function to parse each feed and extract relevant information
def parse_rss(feed_url):
    feed = feedparser.parse(feed_url)
    articles = []

    for entry in feed.entries:
        article = {
            'title': entry.get('title', ''),
            'link': entry.get('link', ''),
            'published': entry.get('published', ''),
            'description': entry.get('description', entry.get('summary', '')),  # Extract description if available, otherwise use summary
            'source': feed.feed.get('title', 'Unknown Source')
        }
        articles.append(article)

    return articles


In [ ]:
#Loop through each RSS feed, parse it, and collect all articles
all_articles = []

for feed_url in rss_feeds:
    print(f"Parsing feed: {feed_url}")
    articles = parse_rss(feed_url)
    all_articles.extend(articles)

print(f"Total articles fetched: {len(all_articles)}")


Parsing feed: https://www.theage.com.au/rss/national/victoria.xml
Parsing feed: https://www.9news.com.au/rss
Parsing feed: https://www.theguardian.com/rss
Parsing feed: https://7news.com.au/feed
Total articles fetched: 255


In [ ]:
#Organize the collected data into a structured format using pandas.

df = pd.DataFrame(all_articles)

# Optional: Convert published date to datetime object
df['published'] = pd.to_datetime(df['published'], errors='coerce')

# Display the first few rows
print(df.head())


                                               title  \
0         Opposition plays the man, forgets the ball   
1  ‘Huge rise in antisemitism’: Top cop says thou...   
2  Factory fire sends smoke across Melbourne, int...   
3  AFL legend Dipper’s son jailed over $140,000 f...   
4  Weeping mourners farewell ‘The Punisher’ in go...   

                                                link  \
0  https://www.theage.com.au/national/victoria/op...   
1  https://www.theage.com.au/national/victoria/hu...   
2  https://www.theage.com.au/national/victoria/li...   
3  https://www.theage.com.au/national/victoria/af...   
4  https://www.theage.com.au/national/victoria/ab...   

                   published  \
0  2025-01-30 20:00:00+11:00   
1  2025-01-30 19:48:08+11:00   
2  2025-01-30 15:34:13+11:00   
3  2025-01-30 14:56:14+11:00   
4  2025-01-30 12:48:28+11:00   

                                         description  \
0  Opposition leader Peter Dutton can make the pr...   
1  Chief Commissioner

<ipython-input-8-9116eca2e158>:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['published'] = pd.to_datetime(df['published'], errors='coerce')


In [ ]:
# Save to CSV
df.to_csv('melbourne_news.csv', index=False)
print("Data saved to melbourne_news.csv")


Data saved to melbourne_news.csv


2. Text Preprocessing

In [ ]:
#Loading and Inspecting the Data

import pandas as pd

# Load the saved CSV file
df = pd.read_csv('melbourne_news.csv')

# Inspect the first few rows
print(df.head())


                                               title  \
0         Opposition plays the man, forgets the ball   
1  ‘Huge rise in antisemitism’: Top cop says thou...   
2  Factory fire sends smoke across Melbourne, int...   
3  AFL legend Dipper’s son jailed over $140,000 f...   
4  Weeping mourners farewell ‘The Punisher’ in go...   

                                                link  \
0  https://www.theage.com.au/national/victoria/op...   
1  https://www.theage.com.au/national/victoria/hu...   
2  https://www.theage.com.au/national/victoria/li...   
3  https://www.theage.com.au/national/victoria/af...   
4  https://www.theage.com.au/national/victoria/ab...   

                   published  \
0  2025-01-30 20:00:00+11:00   
1  2025-01-30 19:48:08+11:00   
2  2025-01-30 15:34:13+11:00   
3  2025-01-30 14:56:14+11:00   
4  2025-01-30 12:48:28+11:00   

                                         description  \
0  Opposition leader Peter Dutton can make the pr...   
1  Chief Commissioner

In [ ]:
# Check for missing values
print(df.isnull().sum())

#fill missing values
df = df.fillna('Unknown')

title            0
link             0
published      215
description      0
source           0
dtype: int64


In [ ]:
# Remove duplicate rows based on the 'title' or 'link'
df = df.drop_duplicates(subset='title', keep='first')

In [ ]:
# Selecting only relevant columns
df = df[['title', 'description', 'link']]


In [ ]:
import re

# Function to clean the text
def clean_text(text):
    # Remove HTML tags and URLs
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove non-alphanumeric characters and lower-case the text
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.lower()

# Apply the cleaning function to the description
df['cleaned_description'] = df['description'].apply(clean_text)


In [ ]:
df.to_csv('cleaned_news_data.csv', index=False)


Categorizing Data with LLM

In [ ]:
from transformers import pipeline

# Load the zero-shot classification model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define candidate labels for your task
candidate_labels = ['violent crime', 'accident', 'other']

# Classify an example text
example_text = "A robbery occurred in Melbourne last night involving multiple suspects."
result = classifier(example_text, candidate_labels)

# Print the results
print(result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


{'sequence': 'A robbery occurred in Melbourne last night involving multiple suspects.', 'labels': ['violent crime', 'other', 'accident'], 'scores': [0.9749300479888916, 0.023876355960965157, 0.001193590578623116]}


In [ ]:
df = pd.read_csv('cleaned_news_data.csv')
df['category'] = df['cleaned_description'].apply(lambda x: classifier(x, candidate_labels)['labels'][0])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df.to_csv('categorized_news_data.csv', index=False)

# Other approach for Zero-shot (less efficient)

Zero shot Classification using DeBERTa

In [ ]:
# from transformers import pipeline

# # Load the DeBERTa-based classification model
# classifier = pipeline("zero-shot-classification", model="microsoft/deberta-v3-large")

# # Define candidate labels for your task
# candidate_labels = ['violent crime', 'accident', 'other']

# # Classify an example text
# example_text = "A robbery occurred in Melbourne last night involving multiple suspects."
# result = classifier(example_text, candidate_labels)

# # Print the results
# print(result)


config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'A robbery occurred in Melbourne last night involving multiple suspects.', 'labels': ['accident', 'violent crime', 'other'], 'scores': [0.33353114128112793, 0.3334712088108063, 0.33299770951271057]}


In [ ]:
# # Apply the classifier to your DataFrame
# df = pd.read_csv('cleaned_news_data.csv')
# df['category'] = df['cleaned_description'].apply(lambda x: classifier(x, candidate_labels)['labels'][0])

In [ ]:
# df.to_csv('categorized_news_data_deberta.csv', index=False)

Zero Shot classification using T5 model


In [ ]:
# from transformers import pipeline

# # Load the T5-based classification model (ensure you use a fine-tuned version for classification tasks)
# classifier = pipeline("zero-shot-classification", model="valhalla/t5-base-qa-qg-hl")

# # Define candidate labels for your task
# candidate_labels = ['violent crime', 'accident', 'other']

# # Classify an example text
# example_text = "A robbery occurred in Melbourne last night involving multiple suspects."
# result = classifier(example_text, candidate_labels)

# # Print the results
# print(result)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at valhalla/t5-base-qa-qg-hl and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


{'sequence': 'A robbery occurred in Melbourne last night involving multiple suspects.', 'labels': ['other', 'violent crime', 'accident'], 'scores': [0.3464841842651367, 0.3303178548812866, 0.32319799065589905]}


In [ ]:
# # Apply the classifier to your DataFrame
# df = pd.read_csv('cleaned_news_data.csv')
# df['category'] = df['cleaned_description'].apply(lambda x: classifier(x, candidate_labels)['labels'][0])

In [ ]:
# df.to_csv('categorized_news_data_t5.csv', index=False)

# Extractiing Crime and Accident Location

In [ ]:
pip install beautifulsoup4 requests

In [ ]:
#Scraping news articles from URL with crime or accident news

import requests
from bs4 import BeautifulSoup

# Load the CSV file with the categorized news data
df = pd.read_csv('categorized_news_data.csv')

# Filter rows where category is 'violent crime' or 'accident'
df = df[df['category'].isin(['violent crime', 'accident'])]

# Function to scrape full article from URL
def scrape_full_article(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the main article content (depends on the website structure)
        article_text = " ".join([p.get_text() for p in soup.find_all('p')])
        return article_text
    except Exception as e:
        return None

# Scrape only articles labeled 'crime' or 'accident'
for index, row in df.iterrows():
    if row['category'] in ['violent crime', 'accident']:
        full_article_text = scrape_full_article(row['link'])
        df.at[index, 'full_text'] = full_article_text


In [ ]:
df.to_csv('location_news_data.csv', index=False)

NameError: name 'df' is not defined

# Working with only open ai and no ner

This may give good results, but it is expensive

In [ ]:
# Load the CSV file with the categorized news data
df = pd.read_csv('test_01.csv')

In [ ]:
#WORKING WITH ONLY OPENAI AND NO NER
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = "your_openai_api_key"

# Function to extract location using OpenAI API
def extract_location_with_openai(full_text):
    # Prepare the prompt
    prompt = (
        f"The following text describes an incident (either a crime or accident). "
        f"Read through the text and identify the exact location where the incident occurred. "
        f"If no location is mentioned or it cannot be determined, return 'None'.\n\n"
        f"Text: {full_text}"
    )

    try:
        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Change to gpt-3.5-turbo if you're using a cheaper model
            messages=[
                {"role": "system", "content": "You are a helpful assistant skilled at extracting the location of the incident from the text. Your response should only include the name of that place"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,  # Limit the response length
            temperature=0  # Reduce randomness for consistent results
        )
        # Extract location from the response
        location = response['choices'][0]['message']['content'].strip()
        return location
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

# Ensure the DataFrame has a `location` column
if 'location' not in df.columns:
    df['location'] = None

# Apply the OpenAI function to the DataFrame
for index, row in df.iterrows():
    if pd.notna(row['full_text']):  # Ensure the text is not null
        location = extract_location_with_openai(row['full_text'])
        df.at[index, 'location'] = location

# Save or inspect the resulting DataFrame
df.to_csv("updated_data_with_locations.csv", index=False)
print(df[['title', 'location']])  # Print title and location for verification


# HYBRID APPROACH - openai + spacy

using spacy for ner

In [ ]:
#RUNNING NER AGAIN USING SPACY THIS TIME
import pandas as pd

# Load the CSV file with the loacions news data
df = pd.read_csv('location_news_data.csv')

#not required; this tep is already been done
# Filter rows where category is 'violent crime' or 'accident'
filtered_df = df[df['category'].isin(['violent crime', 'accident'])]

# Save the result to a new CSV file (optional)
filtered_df.to_csv("filtered_data.csv", index=False)

filtered_df.head()

,title,description,link,cleaned_description,category,full_text
0,Weeping mourners farewell ‘The Punisher’ in go...,The funeral procession for slain gangster Sam ...,https://www.theage.com.au/national/victoria/ab...,the funeral procession for slain gangster sam ...,violent crime,"We’re sorry, this feature is currently unavail..."
1,Monster v The Punisher: The prison slap heard ...,Sam Abdulrahim’s larger-than-life reputation f...,https://www.theage.com.au/national/victoria/mo...,sam abdulrahims largerthanlife reputation for ...,violent crime,"We’re sorry, this feature is currently unavail..."
2,Second suspected getaway car found after Abdul...,After shooting Sam “The Punisher” Abdulrahim a...,https://www.theage.com.au/national/victoria/se...,after shooting sam the punisher abdulrahim and...,violent crime,"We’re sorry, this feature is currently unavail..."
3,Charges dropped against bar patron who confron...,Benjamin Squire had been facing jail time afte...,https://www.theage.com.au/national/victoria/ch...,benjamin squire had been facing jail time afte...,violent crime,"We’re sorry, this feature is currently unavail..."
4,Multiple feared dead after jet collides with U...,<p>T﻿he plane had 64 people on board when it c...,https://www.9news.com.au/world/washington-dc-a...,the plane had 64 people on board when it crash...,accident,Property News: Robin Williams's former San Fra...


In [5]:
import spacy
import pandas as pd
from geopy.geocoders import Nominatim


# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize geocoder
geolocator = Nominatim(user_agent="geo_filter")

# Function to extract locations using spaCy
def extract_locations(text):
    if pd.isna(text):  # Handle missing values
        return []
    doc = nlp(text)
    locations = list(set(ent.text for ent in doc.ents if ent.label_ in ("GPE", "LOC")))
    return locations

# Function to check if a location is in Australia
def is_australian_location(location):
    try:
        geo_info = geolocator.geocode(location, timeout=10)
        if geo_info and "Australia" in geo_info.address:
            return True
    except:
        pass
    return False

# Function to filter out non-Australian locations
def filter_australian_locations(locations):
    return [loc for loc in locations if is_australian_location(loc)]

# Load dataset
df = pd.read_csv("location_news_data.csv")

# Extract locations using spaCy
df["locations"] = df["full_text"].apply(extract_locations)

# Filter out non-Australian locations
df["filtered_locations"] = df["locations"].apply(filter_australian_locations)

# Save the updated dataset
df.to_csv("filtered_dataset.csv", index=False)

print("Filtered locations added!")


Filtered locations added!


# Setting up open ai


In [6]:
import openai
import pandas as pd
import ast  # To safely convert string representation of lists

# Set your OpenAI API key
openai.api_key = ""

# Load the updated DataFrame
df = pd.read_csv("filtered_dataset.csv")

# Function to refine locations using OpenAI
def refine_location_with_openai(ner_locations, full_text):
    if not ner_locations:  # Check if the list is empty
        return None

    # Convert list to a comma-separated string
    ner_locations_str = ", ".join(ner_locations)

    # Prepare the OpenAI prompt
    prompt = (
        f"The following article mentions these locations: {ner_locations_str}. "
        f"Which location is most likely the site of the crime or accident? "
        f"If no location is relevant, return 'None'.\n\n"
        f"Article Text: {full_text}"
    )

    try:
        # Call the OpenAI GPT model
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant specialized in extracting the most relevant locations from news articles. Your response should only include the name of the place where the incident took place in a format that is easier for geopy to plot on the map, as I will be using it to put points on a map."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            temperature=0
        )
        # Extract and return the refined location
        refined_location = response['choices'][0]['message']['content'].strip()
        return None if refined_location.lower() == "none" else refined_location
    except Exception as e:
        print(f"Error with OpenAI API: {e}")
        return None

# Ensure 'filtered_locations' is correctly read as a list (not a string)
df["filtered_locations"] = df["filtered_locations"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Apply the OpenAI refinement function to the DataFrame
df["refined_location"] = df.apply(
    lambda row: refine_location_with_openai(row["filtered_locations"], row["full_text"]) if pd.notna(row["full_text"]) else None,
    axis=1
)

# Save the final updated DataFrame
df.to_csv("final_location_dataset.csv", index=False)

# Preview the result
print(df[["full_text", "filtered_locations", "refined_location"]].head())


                                           full_text  \
0  We’re sorry, this feature is currently unavail...   
1  We’re sorry, this feature is currently unavail...   
2  We’re sorry, this feature is currently unavail...   
3  We’re sorry, this feature is currently unavail...   
4  We’re sorry, this feature is currently unavail...   

                 filtered_locations         refined_location  
0  [Victoria, Melbourne, Truganina]                Truganina  
1           [Mount Clear, Ballarat]  Mount Clear in Ballarat  
2                        [Victoria]                   Echuca  
3    [Bendigo, Mildura, Red Cliffs]                  Mildura  
4                        [Victoria]                Melbourne  


# Locating on Map

In [7]:
!pip install geopandas folium geopy

In [8]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium

# Load the dataset
df = pd.read_csv("final_location_dataset.csv")

# Initialize geolocator
geolocator = Nominatim(user_agent="location_mapper")

# Define the bounding box for Victoria, Australia (lat_min, lat_max, lon_min, lon_max)
victoria_bounds = {
    "lat_min": -39.1597,  # Southernmost point (e.g., Wilsons Promontory)
    "lat_max": -33.9806,  # Northernmost point (near the border with New South Wales)
    "lon_min": 140.9617,  # Westernmost point (near the South Australia border)
    "lon_max": 149.9760   # Easternmost point (e.g., near the border with New South Wales)
}

# Define a function to check if a point lies within the bounding box
def is_within_bounds(lat, lon, bounds):
    return (bounds["lat_min"] <= lat <= bounds["lat_max"]) and (bounds["lon_min"] <= lon <= bounds["lon_max"])

# Extend the get_coordinates function
def get_coordinates(location):
    try:
        if pd.isna(location):  # Handle missing values
            return None, None
        # Geocode the location, appending "Victoria, Australia" for broader search
        # Added a retry mechanism with a delay
        for i in range(3):  # Try up to 3 times
            try:
                location_obj = geolocator.geocode(location + ", Victoria, Australia", timeout=10) # Increased timeout to 10 seconds
                if location_obj:
                    lat, lon = location_obj.latitude, location_obj.longitude
                    # Check if the coordinates are within Melbourne bounds
                    if is_within_bounds(lat, lon, victoria_bounds):
                        return lat, lon
                return None, None  # Location not found or outside bounds
            except (GeocoderTimedOut, GeocoderUnavailable):
                print(f"Geocoding failed for '{location}', retrying in 5 seconds...")
                time.sleep(5)  # Wait 5 seconds before retrying
        return None, None  # Failed after multiple retries
    except Exception as e:
        print(f"Unexpected error: {e}")  # Log unexpected errors
        return None, None


# Apply the function to the 'locations' column
df[['latitude', 'longitude']] = df['refined_location'].apply(lambda loc: pd.Series(get_coordinates(loc)))

# Filter rows with valid coordinates
df = df.dropna(subset=['latitude', 'longitude'])

# Create a map centered on Melbourne
melbourne_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)

# Add points to the map
for _, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=row['refined_location']).add_to(melbourne_map)

# Save the map as an HTML file
melbourne_map.save("melbourne_outer_suburbs_map.html")

# Display the map (optional, requires a Jupyter environment)
melbourne_map


In [ ]:
df.head()

,title,description,link,cleaned_description,category,full_text,locations
0,Second major Victorian hospital set to slash j...,The Royal Melbourne Hospital has joined Alfred...,https://www.theage.com.au/national/victoria/se...,the royal melbourne hospital has joined alfred...,other,NaN,NaN
1,Homicide squad at home in Melbourne’s south-ea...,A crime scene has been established at a home i...,https://www.theage.com.au/national/victoria/tw...,a crime scene has been established at a home i...,violent crime,"We’re sorry, this feature is currently unavail...",NaN
2,"Record number of children restrained, secluded...",Almost 2700 students were restrained or seclud...,https://www.theage.com.au/national/victoria/re...,almost 2700 students were restrained or seclud...,other,NaN,NaN
3,‘Danger to women’: Merri Creek rapist loses bi...,Joel Russo is serving a 20-year jail term for ...,https://www.theage.com.au/national/victoria/da...,joel russo is serving a 20year jail term for a...,violent crime,"We’re sorry, this feature is currently unavail...",NaN
4,Scammed and stranded in the city – by predator...,A night out at a concert; time to catch a cab ...,https://www.theage.com.au/national/victoria/sc...,a night out at a concert time to catch a cab h...,other,NaN,NaN


# Trying classification with clustering techniques (not required)

In [ ]:
#!pip install sentence_transformers

In [ ]:
# import pandas as pd
# from sklearn.cluster import KMeans
# from sentence_transformers import SentenceTransformer
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

# # Load the CSV file with the cleaned and normalized text
# df = pd.read_csv('cleaned_news_data.csv')

# # Initialize sentence-transformer for embedding text data
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# # Convert news descriptions into embeddings
# embeddings = model.encode(df['normalized_description'].tolist())

# # Perform K-Means clustering
# num_clusters = 5  # You can change the number of clusters based on your preference
# kmeans = KMeans(n_clusters=num_clusters, random_state=0)
# df['cluster'] = kmeans.fit_predict(embeddings)

# # Save the results to a new CSV file
# df.to_csv('clustered_news.csv', index=False)
# print("News articles clustered and saved to 'clustered_news.csv'")

# # Optional: Visualize the clusters using PCA (for dimensionality reduction)
# pca = PCA(n_components=2)
# reduced_embeddings = pca.fit_transform(embeddings)

# plt.figure(figsize=(10, 7))
# plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=df['cluster'], cmap='viridis')
# plt.title('Clusters of News Articles')
# plt.show()


Elbow Method for Finding Optimal Clusters:

In [ ]:
# from sklearn.metrics import silhouette_score

# def find_optimal_clusters(embeddings, max_k):
#     iters = range(2, max_k+1)
#     sse = []
#     silhouette_scores = []

#     for k in iters:
#         kmeans = KMeans(n_clusters=k, random_state=0)
#         kmeans.fit(embeddings)
#         sse.append(kmeans.inertia_)  # Sum of squared distances
#         silhouette_scores.append(silhouette_score(embeddings, kmeans.labels_))

#     plt.figure(figsize=(10, 7))
#     plt.plot(iters, sse, marker='o')
#     plt.title('Elbow Method for Optimal Clusters')
#     plt.xlabel('Number of clusters')
#     plt.ylabel('SSE')
#     plt.show()

#     plt.figure(figsize=(10, 7))
#     plt.plot(iters, silhouette_scores, marker='o')
#     plt.title('Silhouette Score for Optimal Clusters')
#     plt.xlabel('Number of clusters')
#     plt.ylabel('Silhouette Score')
#     plt.show()

# # Run the Elbow Method to find the optimal number of clusters
# find_optimal_clusters(embeddings, 10)


One Shot Classification Using BERT

In [ ]:
# !pip install openai

In [ ]:
# import openai
# import pandas as pd

# # Load your CSV file with news descriptions
# df = pd.read_csv('cleaned_news_data.csv')

# # Define your categories for classification
# categories = ['Crime', 'Government', 'Business', 'violence', 'Science', 'Culture']

# # Initialize OpenAI API
# openai.api_key = ""

# # Function to generate GPT-3 prompt for zero-shot classification
# def classify_article_with_gpt3(article, categories):
#     prompt = f"Classify the following news article into one of these categories: {', '.join(categories)}.\n\nArticle: {article}\n\nCategory:"

#     response = openai.Completion.create(
#         engine="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": prompt}
#         ],
#         prompt=prompt,
#         max_tokens=10,
#         n=1,
#         stop=None,
#         temperature=0  # Lower temperature for deterministic output
#     )

#     category = response.choices[0].text.strip()
#     return category

# # Apply classification on each article
# df['gpt3_category'] = df['normalized_description'].apply(lambda x: classify_article_with_gpt3(x, categories))

# # Save the results to CSV
# df.to_csv('gpt3_classified_news.csv', index=False)
# print("GPT-3 classification complete. Results saved to 'gpt3_classified_news.csv'")
